pip install transformers datasets peft accelerate bitsandbytes
cool link: https://towardsdatascience.com/lora-intuitively-and-exhaustively-explained-e944a6bff46b/


you must login via hugging face first:
1. v terminal: huggingface-cli login
2. generate your own token on https://huggingface.co/settings/tokens
    - Click "New token"
    - Choose a name (e.g., llama3-training)
    - Select role: Read
    - Click Generate
    - copy the token
3. paste the token in terminal

\\
\\

Meta-Llama-3-8B (https://huggingface.co/meta-llama/Meta-Llama-3-8B ali https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) --> 13GB disk space, 16-24GB ram \

Meta-Llama-2-7B (https://huggingface.co/meta-llama/Llama-2-7b-hf) --> 15GB disk space, 16-24GB ram




example json:
[
  { "input": "Report 1: Crash on I-95. Report 2: Delays near downtown. Report 2: Crash on I-95. Report 2: Delays near downtown.", 
    "output": "I-95 crash and downtown congestion causing major delays." },

  { "input": "Report: Accident cleared on Route 1. Flow normal. Report 2: Crash on I-95. Report 2: Delays near downtown.", 
    "output": "Traffic back to normal on Route 1 after earlier accident." },

  ...
]

In [8]:
# quick preprocessing to add StrippedContent to output, without this part:
# Prometne informacije   \t   29. 08. 2022\t  12.00  1. in 3. program\n\nPodatki o prometu.)

def strip_output_content(data):
    for entry in data:
        output_content = entry["output"]["Content"]
        
        # Split into chunks using double newline
        parts = output_content.split("\n\n")

        # Keep everything after the first two chunks
        if len(parts) > 2:
            stripped = "\n\n".join(parts[2:])
        else:
            stripped = output_content  # fallback: keep as-is

        # Add new field
        entry["output"]["StrippedContent"] = stripped

    return data


import json

# Load the JSON
with open("/Users/petrakuralt/Downloads/input_output_all_data_2024.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Add stripped content
data = strip_output_content(data)

# Save back to file (optional)
with open("input_output_all_data_2024_with_stripped.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)




In [ ]:
# test dostop do llame
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

# Get token from environment
hf_token = os.getenv("HUGGINGFACE_TOKEN")

# Log in
login(token=hf_token)

model_id = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True)

KeyboardInterrupt: 

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

# === Config ===
# model_name = "meta-llama/Llama-2-7b-hf"
model_name = "meta-llama/Meta-Llama-3-8B"
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"


dataset_path = "your_dataset.json"  # or Hugging Face dataset path
output_dir = "./lora-results"

# === Load tokenizer and model ===
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # needed for training

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True
)

# # === Apply LoRA ===
# lora_config = LoraConfig(
#     r=8,            # the rank of the A and B matrices
#     lora_alpha=16,  # scaling factor, and by default it should be equal to r
#     target_modules=["q_proj", "v_proj"],  # which parts of the model LoRA should modify (the projection layers for query and value)
#     lora_dropout=0.1,       # hides inputs to prevent the model from overfitting
#     bias="none",            # are we adding bias to the LoRA layers or just weights
#     task_type=TaskType.CAUSAL_LM,  # decoder-only LM
# )

# model = get_peft_model(model, lora_config)
# model.print_trainable_parameters()

# # === Load dataset ===
# data = load_dataset("json", data_files=dataset_path, split="train")

# # === Preprocess --> converts raw input,output pairs into tokenized tensors ===
# # "Summarize traffic news: report 1: Na dolenjski avtocesti zastoj... .Summary: <studentski report>"
# def preprocess(example):
#     prompt = f"Summarize traffic news:\n{example['input']}\nSummary:"
#     output = example['output']
#     full_text = prompt + " " + output
#     return tokenizer(full_text, truncation=True, padding="max_length", max_length=512) # Truncates longer examples to fit max 512 tokens. Pads shorter ones to exactly 512 tokens.

# # Iterate over every example in dataset and apply the preprocess function
# tokenized_data = data.map(preprocess)

# # === Training setup ===
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=4,
#     logging_steps=10,       # log loss every 10 steps
#     num_train_epochs=3,     # number of epochs to train, baje je 3 ze dost za loro 
#     save_strategy="epoch",  # save model every epoch
#     learning_rate=2e-4,
#     fp16=True,           # use 16-bit floating point precision
#     report_to="none",    # disable logging to other services
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_data,
#     tokenizer=tokenizer,
#     data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
# )

# # === Train ===
# trainer.train()

# # === Save ===
# model.save_pretrained(output_dir)
# tokenizer.save_pretrained(output_dir)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
# Nesreče Na cesti Strunjan - Lucija promet poteka izmenično enosmerno pri Strunjanu. Ovire Okvarjeno tovorno vozilo na mariborski vzhodni obvoznici ovira promet med pokritim vkopom Malečnik in predorom Vodole proti Mariboru. Na primorski avtocesti zaradi okvare vozila zaprt vozni pas na viaduktu Črni Kal proti Kopru. Zastoji Zaradi popoldanske prometne konice je upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo iz mestnih središč. Zastoji zaradi del: - Na štajerski avtocesti pred počivališčem Polskava proti Ljubljani, zamuda 5 minut. - Na cesti Postojna - Pivka v obe pri Prestranku in Petelinjah. Delovne zapore Danes ponoči med 18. in 5. uro bo na gorenjski avtocesti promet skozi predor Karavanke potekal izmenično enosmerno. Na cesti Nova Gorica - Rožna Dolina bo zaprt predor Panovec, danes od 17. ure, do 30. 10 do 15. ure. Cesta Kamnik - Stahovica - Črnivec bo zaprta zaradi asfaltiranja pri Podstudencu vsak dan med 7. in 17. uro do 30. 10., obvoz je urejen. Glavna cesta Litija - Zagorje bo zaprta pri Šklendrovcu v sredo, 30. 10. , predvidoma med 8. in 17. uro. Več o delovnih zaporah v prometni napovedi . Tovorna vozila V četrtek, 31. 10. in v petek, 1. 11., med 8. in 22. uro velja omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7.5 tone.",
# Nesreče Na cesti Strunjan - Lucija promet poteka izmenično enosmerno pri Strunjanu. Ovire Okvarjeno tovorno vozilo na mariborski vzhodni obvoznici ovira promet med pokritim vkopom Malečnik in predorom Vodole proti Mariboru. Na primorski avtocesti zaradi okvare vozila zaprt vozni pas na viaduktu Črni Kal proti Kopru. Zastoji Zaradi popoldanske prometne konice je upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo iz mestnih središč. Zastoji zaradi del: - Na štajerski avtocesti pred počivališčem Polskava proti Ljubljani, zamuda 5 minut. - Na cesti Postojna - Pivka v obe pri Prestranku in Petelinjah. Na cestah Lucija - Strunjan in Izola - Lucija. Delovne zapore Danes ponoči med 18. in 5. uro bo na gorenjski avtocesti promet skozi predor Karavanke potekal izmenično enosmerno. Na cesti Nova Gorica - Rožna Dolina bo zaprt predor Panovec, danes od 17. ure, do 30. 10 do 15. ure. Cesta Kamnik - Stahovica - Črnivec bo zaprta zaradi asfaltiranja pri Podstudencu vsak dan med 7. in 17. uro do 30. 10., obvoz je urejen. Glavna cesta Litija - Zagorje bo zaprta pri Šklendrovcu v sredo, 30. 10. , predvidoma med 8. in 17. uro. Več o delovnih zaporah v prometni napovedi . Tovorna vozila V četrtek, 31. 10. in v petek, 1. 11., med 8. in 22. uro velja omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7.5 tone.",
# Nesreče Na cesti Strunjan - Lucija promet poteka izmenično enosmerno pri Strunjanu. Ovire Na primorski avtocesti zaradi okvare vozila zaprt vozni pas na viaduktu Črni Kal proti Kopru. Zastoji Zaradi popoldanske prometne konice je upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo iz mestnih središč. Zastoji zaradi del: - Na štajerski avtocesti pred počivališčem Polskava proti Ljubljani, zamuda 5 minut. - Na cesti Postojna - Pivka v obe pri Prestranku in Petelinjah. Na cestah Lucija - Strunjan in Izola - Lucija. Delovne zapore Danes ponoči med 18. in 5. uro bo na gorenjski avtocesti promet skozi predor Karavanke potekal izmenično enosmerno. Na cesti Nova Gorica - Rožna Dolina bo zaprt predor Panovec, danes od 17. ure, do 30. 10 do 15. ure. Cesta Kamnik - Stahovica - Črnivec bo zaprta zaradi asfaltiranja pri Podstudencu vsak dan med 7. in 17. uro do 30. 10., obvoz je urejen. Glavna cesta Litija - Zagorje bo zaprta pri Šklendrovcu v sredo, 30. 10. , predvidoma med 8. in 17. uro. Več o delovnih zaporah v prometni napovedi . Tovorna vozila V četrtek, 31. 10. in v petek, 1. 11., med 8. in 22. uro velja omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7.5 tone.",
# Nesreče Na cesti Strunjan - Lucija promet poteka izmenično enosmerno pri Strunjanu, nastal je zastoj v obe smeri. Ovire Na primorski avtocesti zaradi okvare vozila zaprt vozni pas na viaduktu Črni Kal proti Kopru. Zastoji Zaradi popoldanske prometne konice je upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo iz mestnih središč. Zastoji zaradi del: - Na štajerski avtocesti pred počivališčem Polskava proti Ljubljani, zamuda 5 minut. - Na cesti Postojna - Pivka v obe pri Prestranku in Petelinjah. Delovne zapore Danes ponoči med 18. in 5. uro bo na gorenjski avtocesti promet skozi predor Karavanke potekal izmenično enosmerno. Na cesti Nova Gorica - Rožna Dolina bo zaprt predor Panovec, danes od 17. ure, do 30. 10 do 15. ure. Cesta Kamnik - Stahovica - Črnivec bo zaprta zaradi asfaltiranja pri Podstudencu vsak dan med 7. in 17. uro do 30. 10., obvoz je urejen. Glavna cesta Litija - Zagorje bo zaprta pri Šklendrovcu v sredo, 30. 10. , predvidoma med 8. in 17. uro. Več o delovnih zaporah v prometni napovedi . Tovorna vozila V četrtek, 31. 10. in v petek, 1. 11., med 8. in 22. uro velja omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7.5 tone.",
# Nesreče Na cesti Izola - Lucija promet poteka izmenično enosmerno med krožiščema pri Strunjanu in Valeti, nastal je zastoj v obe smeri. Ovire Na primorski avtocesti zaradi okvare vozila zaprt vozni pas na viaduktu Črni Kal proti Kopru. Zastoji Zaradi popoldanske prometne konice je upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo iz mestnih središč. Zastoji zaradi del: - Na štajerski avtocesti pred počivališčem Polskava proti Ljubljani, zamuda 5 minut. - Na cesti Postojna - Pivka v obe pri Prestranku in Petelinjah. Delovne zapore Danes ponoči med 18. in 5. uro bo na gorenjski avtocesti promet skozi predor Karavanke potekal izmenično enosmerno. Na cesti Nova Gorica - Rožna Dolina bo zaprt predor Panovec, danes od 17. ure, do 30. 10 do 15. ure. Cesta Kamnik - Stahovica - Črnivec bo zaprta zaradi asfaltiranja pri Podstudencu vsak dan med 7. in 17. uro do 30. 10., obvoz je urejen. Glavna cesta Litija - Zagorje bo zaprta pri Šklendrovcu v sredo, 30. 10. , predvidoma med 8. in 17. uro. Več o delovnih zaporah v prometni napovedi . Tovorna vozila V četrtek, 31. 10. in v petek, 1. 11., med 8. in 22. uro velja omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7.5 tone.",
# Nesreče Na cesti Izola - Lucija promet poteka izmenično enosmerno med krožiščema pri Strunjanu in Valeti, nastal je zastoj v obe smeri. Na ljubljanski južni obvoznici je oviran promet pred priključkom Rudnik proti razcepu Malence. Ovire Na primorski avtocesti zaradi okvare vozila zaprt vozni pas na viaduktu Črni Kal proti Kopru. Zastoji Zaradi popoldanske prometne konice je upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo iz mestnih središč. Zastoji zaradi del: - Na štajerski avtocesti pred počivališčem Polskava proti Ljubljani, zamuda 5 minut. - Na cesti Postojna - Pivka v obe pri Prestranku in Petelinjah. Delovne zapore Danes ponoči med 18. in 5. uro bo na gorenjski avtocesti promet skozi predor Karavanke potekal izmenično enosmerno. Na cesti Nova Gorica - Rožna Dolina bo zaprt predor Panovec, danes od 17. ure, do 30. 10 do 15. ure. Cesta Kamnik - Stahovica - Črnivec bo zaprta zaradi asfaltiranja pri Podstudencu vsak dan med 7. in 17. uro do 30. 10., obvoz je urejen. Glavna cesta Litija - Zagorje bo zaprta pri Šklendrovcu v sredo, 30. 10. , predvidoma med 8. in 17. uro. Več o delovnih zaporah v prometni napovedi . Tovorna vozila V četrtek, 31. 10. in v petek, 1. 11., med 8. in 22. uro velja omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7.5 tone.",
# Nesreče Na cesti Izola - Lucija promet poteka izmenično enosmerno med krožiščema pri Strunjanu in Valeti, nastal je zastoj v obe smeri. Na ljubljanski južni obvoznici je oviran promet pred priključkom Rudnik proti razcepu Malence. Ovire Na primorski avtocesti zaradi okvare vozila zaprt vozni pas na viaduktu Črni Kal proti Kopru. Zastoji Zaradi popoldanske prometne konice je upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo iz mestnih središč. Zastoji zaradi del: - Na štajerski avtocesti pred počivališčem Polskava proti Ljubljani, zamuda 5 minut. - Na ljubljanski južni obvoznici med priključkoma Rudnik in Ljubljana center v obe smeri. - Na cesti Postojna - Pivka v obe pri Prestranku in Petelinjah. Delovne zapore Danes ponoči med 18. in 5. uro bo na gorenjski avtocesti promet skozi predor Karavanke potekal izmenično enosmerno. Na cesti Nova Gorica - Rožna Dolina bo zaprt predor Panovec, danes od 17. ure, do 30. 10 do 15. ure. Cesta Kamnik - Stahovica - Črnivec bo zaprta zaradi asfaltiranja pri Podstudencu vsak dan med 7. in 17. uro do 30. 10., obvoz je urejen. Glavna cesta Litija - Zagorje bo zaprta pri Šklendrovcu v sredo, 30. 10. , predvidoma med 8. in 17. uro. Več o delovnih zaporah v prometni napovedi . Tovorna vozila V četrtek, 31. 10. in v petek, 1. 11., med 8. in 22. uro velja omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7.5 tone.",
# Nesreče Na cesti Izola - Lucija promet poteka izmenično enosmerno med krožiščema pri Strunjanu in Valeti, nastal je zastoj v obe smeri. Na ljubljanski južni obvoznici je oviran promet pred priključkom Rudnik proti razcepu Malence. Ovire Na primorski avtocesti je zaradi okvare vozila oviran promet na viaduktu Črni Kal proti Kopru. Zastoji Zaradi popoldanske prometne konice je upočasnjen promet na mestnih obvoznicah in cestah, ki vodijo iz mestnih središč. Zastoji zaradi del: - Na štajerski avtocesti pred počivališčem Polskava proti Ljubljani, zamuda 5 minut. - Na ljubljanski južni obvoznici med priključkoma Rudnik in Ljubljana center v obe smeri. - Na cesti Postojna - Pivka v obe pri Prestranku in Petelinjah. Delovne zapore Danes ponoči med 18. in 5. uro bo na gorenjski avtocesti promet skozi predor Karavanke potekal izmenično enosmerno. Na cesti Nova Gorica - Rožna Dolina bo zaprt predor Panovec, danes od 17. ure, do 30. 10 do 15. ure. Cesta Kamnik - Stahovica - Črnivec bo zaprta zaradi asfaltiranja pri Podstudencu vsak dan med 7. in 17. uro do 30. 10., obvoz je urejen. Glavna cesta Litija - Zagorje bo zaprta pri Šklendrovcu v sredo, 30. 10. , predvidoma med 8. in 17. uro. Več o delovnih zaporah v prometni napovedi . Tovorna vozila V četrtek, 31. 10. in v petek, 1. 11., med 8. in 22. uro velja omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7.5 tone.",

In [9]:
dataset_path = "/Users/petrakuralt/Downloads/input_output_all_data_2022_with_stripped.json"
with open(dataset_path, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

#print how many entries
print(f"Total entries in dataset: {len(raw_data)}")
# subset_data = raw_data[:30]

Total entries in dataset: 8259


In [ ]:

Na štajerski avtocesti: - med priključkoma Vransko in Šentrupert v obe smeri zaprt prehitevalni pas; - 
na razcepu Slivnica iz smeri Maribora proti Ptuju zaprt vozni pas. Na štajerski avtocesti: - 
med priključkoma Vransko in Šentrupert v obe smeri zaprt prehitevalni pas, nastal je zastoj; - 
na razcepu Slivnica iz smeri Maribora proti Ptuju zaprt vozni pas. Na dolenjski avtocesti med 
priključkoma Trebnje zahod in Bič proti Ljubljani zaprt počasni pas zaradi okvare tovornega vozila.
Na ljubljanski zahodni obvoznici na razcepu Kozarje iz smeri Brda proti Viču spolzko vozišče. 
Na štajerski avtocesti: - med priključkoma Šentrupert in Vransko proti Ljubljani zaprt prehitevalni pas, 
nastal je zastoj; - na razcepu Slivnica iz smeri Maribora proti Ptuju zaprt vozni pas. Na štajerski avtocesti: - 
med priključkoma Šentrupert in Vransko proti Ljubljani zaprt prehitevalni pas, nastal je zastoj. Na štajerski avtocesti 
proti Ljubljani med priključkoma Šentrupert in Vransko zaprt prehitevalni pas, pred predorom Jasovnik pa je oviran promet
na odstavnem pasu. Na štajerski avtocesti proti Ljubljani med priključkoma Šentrupert in Vransko zaprt prehitevalni pas. 
Po podatkih ARSO bodo do večera padavine zajele vso Slovenijo. Meja sneženja se bo do večera spustila do nižin. Računajte 
na daljši čas potovanja, na pot pa se odpravite samo z ustrezno zimsko opremo. Od 30. decembra je v veljavi sprememba omejitve 
za tovorna vozila nad 7,5 ton. Več. Čakalna doba je na Obrežju in Gruškovju. Na prelazih Pavličevo sedlo, Jezerski vrh, Ljubelj
in Korensko sedlo so obvezne verige na avstrijski strani. Prelaz Vršič je zaprt.
